In [ ]:
# !pip install spacy
# !python -m spacy download en_core_web_sm

In [1]:
import sys
import logging

nblog = open("nb_clean.log", "a+")
sys.stdout.echo = nblog
sys.stderr.echo = nblog

get_ipython().log.handlers[0].stream = nblog
get_ipython().log.setLevel(logging.INFO)

%autosave 5

Autosaving every 5 seconds


In [2]:
import spacy
nlp = spacy.load("en_core_web_sm")
import pandas as pd
from tqdm import tqdm
import nltk
import re
import ast
from nltk.corpus import stopwords
from collections import Counter, defaultdict
nltk.download('stopwords')
import glob
from textCleaning import parse_stopwords, get_file_names, extract_year_ipc
import pandas as pd

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/edgarlanoue/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/edgarlanoue/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
import importlib
import textCleaning
importlib.reload(textCleaning)
from textCleaning import parse_stopwords, get_file_names, extract_year_ipc, Vocab

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/edgarlanoue/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
# path = "C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/"
path = "/home/edgarlanoue/data/"

In [9]:
technet = pd.read_csv(path+"Technet/clean_vocab.csv")
tn_lemm = pd.read_csv(path+"Technet/lemmatized_technet.csv")

In [10]:

file_path = "/home/edgarlanoue/brevetNLP/technet/stopwords.txt" #path+"PatentNovelty/technet/stopwords.txt" 
sW_dict = parse_stopwords(file_path)

# Access each list by its label
stopwords_nltk = sW_dict.get("Stopwords NLTK", [])
stopwords_ustpo = sW_dict.get("Stopwords USTPO", [])
stopwords_tn_technical = sW_dict.get("Stopwords TN_technical", [])
to_add = sW_dict.get("toAdd", [])
new_sw = sW_dict.get("newSW", [])

# Print the lists to verify
print("length= ", len(stopwords_nltk), ", ", "Stopwords NLTK:", stopwords_nltk)
print("length= ", len(stopwords_ustpo), ", ", "Stopwords USTPO:", stopwords_ustpo)
print("length= ", len(stopwords_tn_technical), ", ", "Stopwords TN_technical:", stopwords_tn_technical)
print("length= ", len(to_add), ", ", "toAdd:", to_add)
print("length= ", len(new_sw), ", ", "newSW:", new_sw)

stopwords = list(set([item for sublist in sW_dict.values() for item in sublist]))

length=  179 ,  Stopwords NLTK: ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only'

In [11]:
vocab=textCleaning.Vocab(technet=technet, df_lemm=tn_lemm, stopwords=stopwords)
# len(vocab.tn_lemm_filtered) # 845647

In [12]:
path = "/home/edgarlanoue/data/csv/"  #"C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/test/"
pathOutput = "/home/edgarlanoue/data/csvCleaned/" #"C:/Users/edgar/OneDrive/Bureau/Ecole/HEC/A24/BrevetNLP/exemple données/cleaned/"

In [13]:
def cleanCSV(path, pathOutput, yearList, ipcList):
    tE_names = get_file_names(path+"toEval")
    KS_names = get_file_names(path+"KS")
    ES_names = get_file_names(path+"ES")

    tE_set = set([extract_year_ipc(string) for string in tE_names])
    KS_set = set([extract_year_ipc(string) for string in KS_names])
    ES_set = set([extract_year_ipc(string) for string in ES_names])

    assert(tE_set==KS_set==ES_set)

    if yearList != None and ipcList != None:
        yearList = yearList
        ipcList = ipcList
    else:
        yearList = [year for year, ipc in list(tE_set)]
        ipcList = [ipc for year, ipc in list(tE_set)]

    for year in yearList:
        for ipc in ipcList:
            print(f"{year}")
            print(f"    {ipc}")
            
            # tE = pd.read_csv(path+f"exemple données/test/toEval/{year}_{ipc}_patents_toEval.csv")
            # matching_files = glob.glob(path+f"exemple données/test/KS/{year}_*_{ipc}_KS_raw.csv")[0]
            # KS = pd.read_csv(matching_files)
            # matching_files = glob.glob(path+f"exemple données/test/ES/{year}_*_{ipc}_ES_raw.csv")[0]
            # ES = pd.read_csv(matching_files)

            tE = pd.read_csv(path+f"toEval/{year}_{ipc}_patents_toEval.csv")
            matching_files = glob.glob(path+f"KS/{year}_*_{ipc}_KS_raw.csv")[0]
            KS = pd.read_csv(matching_files)
            matching_files = glob.glob(path+f"ES/{year}_*_{ipc}_ES_raw.csv")[0]
            ES = pd.read_csv(matching_files)

            tE_clean = vocab.lemmDF(vocab.cleanDF(tE, type="all"))
            KS_clean = vocab.lemmDF(vocab.cleanDF(KS, type="all"))
            ES_clean = vocab.lemmDF(vocab.cleanDF(ES, type="all"))

            # tE_clean.to_csv(path+f"exemple données/cleaned/tE/{year}_{ipc}_tE_cleaned.csv", index=False)
            # KS_clean.to_csv(path+f"exemple données/cleaned/KS/{year}_{ipc}_KS_cleaned.csv", index=False)
            # ES_clean.to_csv(path+f"exemple données/cleaned/ES/{year}_{ipc}_ES_cleaned.csv", index=False)

            tE_clean.to_csv(pathOutput+f"/tE/{year}_{ipc}_tE_cleaned.csv", index=False)
            KS_clean.to_csv(pathOutput+f"/KS/{year}_{ipc}_KS_cleaned.csv", index=False)
            ES_clean.to_csv(pathOutput+f"/ES/{year}_{ipc}_ES_cleaned.csv", index=False)

    return None

        

In [ ]:
yearList = ['2013', '2014', '2015']
ipcList = ["H01L", "A61B"]

cleanCSV(path, pathOutput, yearList, ipcList)

2013
    H01L
Clean


  0%|          | 0/20359 [00:00<?, ?it/s]

In [ ]:
yearList = ['2012', '2013', '2014', '2015', '2016']
ipcList = ["G01N", "B60L", "E21B", "F03D", "H04W", "C07D", "D07B", "B32B"]

cleanCSV(path, pathOutput, yearList, ipcList)

In [14]:
path_tE = path+"toEval" # exemple données/test/
path_KS = path+"KS" #exemple données/test/
path_ES = path+"ES" #exemple données/test/

tE_names = get_file_names(path_tE)
KS_names = get_file_names(path_KS)
ES_names = get_file_names(path_ES)

tE_set = set([extract_year_ipc(string) for string in tE_names])
KS_set = set([extract_year_ipc(string) for string in KS_names])
ES_set = set([extract_year_ipc(string) for string in ES_names])

assert(tE_set==KS_set==ES_set)

yearList = sorted(list(set([year for year, ipc in list(tE_set)])))
ipcList = list(set([ipc for year, ipc in list(tE_set)]))

In [15]:
print(yearList)
print(ipcList)

['2011', '2012', '2013', '2014', '2015', '2016']
['G01N', 'E21B', 'F03D', 'B60L', 'B32B', 'G06F', 'H04W', 'C07D', 'D07B', 'A61B', 'H01L']


In [8]:
yearList = ['2012', '2013', '2014', '2015', '2016']
ipcList = ["A61B", "H01L"]

In [9]:
yearList = ['2016']#, '2013', '2014', '2015', '2016']
ipcList = ["H01L"]

In [10]:
for year in yearList:
    for ipc in ipcList:
        print(f"{year}")
        print(f"    {ipc}")
        tE = pd.read_csv(path+f"toEval/{year}_{ipc}_patents_toEval.csv")
        matching_files = glob.glob(path+f"KS/{year}_*_{ipc}_KS_raw.csv")[0]
        KS = pd.read_csv(matching_files)
        matching_files = glob.glob(path+f"ES/{year}_*_{ipc}_ES_raw.csv")[0]
        ES = pd.read_csv(matching_files)

2016
    H01L


In [28]:
# tE_clean = vocab.lemmDF(vocab.cleanDF(tE, type="all"))

In [53]:
tE_clean = vocab.lemmDF(vocab.cleanDF(tE, type="all"))
KS_clean = vocab.lemmDF(vocab.cleanDF(KS, type="all"))
ES_clean = vocab.lemmDF(vocab.cleanDF(ES, type="all"))

Clean


100%|██████████| 40960/40960 [01:45<00:00, 389.05it/s]


Lemmatize


100%|██████████| 40960/40960 [00:06<00:00, 6396.51it/s]


Clean


...

100%|██████████| 173135/173135 [00:28<00:00, 6072.73it/s]


Clean


100%|██████████| 396708/396708 [23:22<00:00, 282.86it/s] 


Lemmatize


100%|██████████| 396708/396708 [01:18<00:00, 5081.83it/s]


In [33]:
# tE_clean.to_csv(path+f"/home/edgarlanoue/data/csvCleaned/tE/{year}_{ipc}_tE_cleaned.csv", index=False)

In [37]:
pathOutput

'/home/edgarlanoue/data/csvCleaned/'

In [54]:
tE_clean.to_csv(pathOutput+f"/tE/{year}_{ipc}_tE_cleaned.csv", index=False)
KS_clean.to_csv(pathOutput+f"/KS/{year}_{ipc}_KS_cleaned.csv", index=False)
ES_clean.to_csv(pathOutput+f"/ES/{year}_{ipc}_ES_cleaned.csv", index=False)

In [211]:
# Vérification
# count_word_frequency_total(clean_KS_H01L_2016) - c'est bon